# Transfer Matrix and Lattice Dilatation Operator for High-Quality Fixed Points in Tensor Network Renormalization Group
## Reproduction of some of the paper results

This notebook reproduces some of the results presented in the paper "Transfer Matrix and Lattice Dilatation Operator for High-Quality Fixed Points in Tensor Network Renormalization Group." Let us start with installing dependencies and ensuring the correct environment is used. 

In [ ]:
using Pkg
Pkg.activate(".")
include("Tools.jl")
include("KrylovTechnical.jl")
include("GaugeFixing.jl");

We are going to reproduce results from Tables 4 and 5 for rotating algorithm. First of all, we need to get the fixed point tensor. We start by finding an initial approximation using the shooting method.

In [ ]:
run(`julia --project Lab/critical_temperature.jl --rotate true`)

In [1]:
chi=30
Jratio=1.0
number_of_initial_steps=23
gilt_pars = Dict(
	"gilt_eps" => 6e-6,
	"cg_chis" => collect(1:chi),
	"cg_eps" => 1e-10,
	"verbosity" => 0,
	"rotate" => false,
)

search_tol = 1.0e-10
relT_low, relT_high, relT = find_critical_temperature(chi, gilt_pars, search_tol, Jratio)
if abs(relT) < 1.e-10
    throw("Critical relT was not found in critical_temperatures/")
end

initialA_pars = Dict("relT" => relT, "Jratio" => Jratio)

A_crit_approximation = trajectory(initialA_pars, number_of_initial_steps, gilt_pars)["A"][end]

PyObject <class 'tensors.symmetrytensors.TensorZ2'>([[15, 15], [15, 15], [15, 15], [15, 15]], qhape=[[1, 0], [1, 0], [1, 0], [1, 0]], qodulus=2, sects={(0, 0, 0, 0): array([[[[ 3.3714190976e-02, -2.7785018343e-03,  1.0784923370e-05, ...,
           1.9369822840e-04,  9.9341712712e-07,  6.4065896604e-08],
         [-3.4285608970e-03,  4.8980533756e-03,  3.6874056754e-03, ...,
          -1.3734503394e-05,  2.6412795619e-05, -7.2674704882e-07],
         [ 2.3845639543e-04,  2.8226890667e-03,  3.4128925350e-03, ...,
           4.5632527997e-05,  2.0426561236e-05, -9.1522412675e-07],
         ...,
         [-1.7217671527e-04,  1.5938680113e-05, -5.8591866394e-05, ...,
          -4.4702466531e-05, -1.3348384979e-05, -3.1930571494e-06],
         [ 1.5246812135e-05, -3.3174984942e-05, -3.1179701710e-05, ...,
          -7.8639426577e-06, -1.1692399373e-05, -2.1264302847e-06],
         [-1.5511600104e-07, -8.2945557313e-06, -7.7490026941e-06, ...,
           5.6015374703e-07,  1.8834822552e-07, 

In [11]:
using TensorOperations
A=A_crit_approximation.to_ndarray();
@tensor env_h_1[-1,-2]:=A[1,2,-1,3]*A[1,2,-2,3];
@tensor env_h_2[-1,-2]:=A[-1,2,3,4]*A[-2,2,3,4];
@tensor env_v_1[-1,-2]:=A[-1,2,3,4]*A[-2,2,3,4];
@tensor env_v_2[-1,-2]:=A[1,2,3,-1]*A[1,2,3,-2];
println((env_h_1-env_h_2 |> norm)/(env_h_1 |> norm))
println((env_v_1-env_v_2 |> norm)/(env_v_1 |> norm))


0.0009493327523196435
0.06052064607701979


In [12]:
Newton_data=deserialize("newton/rotate=true_30_6.0e-6_1.0e-10_jac_approximation_rank=54.data");

In [16]:
A=(Newton_data["A_newton"][14] |> ju_to_py).to_ndarray()
@tensor env_h_1[-1,-2]:=A[1,2,-1,3]*A[1,2,-2,3];
@tensor env_h_2[-1,-2]:=A[-1,2,3,4]*A[-2,2,3,4];
@tensor env_v_1[-1,-2]:=A[-1,2,3,4]*A[-2,2,3,4];
@tensor env_v_2[-1,-2]:=A[1,2,3,-1]*A[1,2,3,-2];
println((env_h_1-env_h_2 |> norm)/(env_h_1 |> norm))
println((env_v_1-env_v_2 |> norm)/(env_v_1 |> norm))

0.05545129839240479
0.019750950266812704


In [2]:
using Serialization
LDO_data=deserialize("DSO/rotate=false_30_6.0e-6_1.0e-10_rotate=(false).data");

In [3]:
sys=LDO_data["eigensystem"];
vals,vecs=sys[1],sys[2];
length(vals)

172

In [4]:
imag.(vals)[2:end] .|> println;

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.2923605302688696e-5
-1.2923605302688696e-5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.9378541655412555e-6
-1.9378541655412555e-6
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
3.4907947250983804e-5
-3.4907947250983804e-5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
9.233746163650205e-6
-9.233746163650205e-6
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [5]:
-log.(2,real.(vals))[2:end] .|> println;

0.12476491130085283
0.999346818305128
1.123617583648248
1.1240849896916894
1.9964378275635681
1.9973776821518097
1.9974583852890535
1.9982078459278068
2.118795218975457
2.1204806988411202
2.1237384650622495
2.990344239988964
2.9908026234572223
2.992753118674738
2.996279032392098
2.9985908670533634
3.1105779880717956
3.1139912938145735
3.1191632368765787
3.1193065496285697
3.1228460901236867
3.123651179333387
3.9842528399964645
3.985354658675925
3.9882707006984663
3.9897057909128315
3.9930010498942523
3.993306378904414
3.9996769433037738
4.000064066989676
4.000280302465395
4.086479987959523
4.113220732147002
4.115398279303323
4.119508345377879
4.123331154288945
4.1238206200786145
4.1238206200786145
4.12681117265033
4.143206239189571
4.98324798884244
4.985418798048549
4.991692666300015
4.9926773727912455
4.99934137358292
5.002794122545913
5.003725846029969
5.003932702927632
5.005392549264444
5.010620260951726
5.012459922656623
5.021242429913222
5.029728414524188
5.097391017705921
5.09739

In [19]:
exact_spectrum(180) .|>println;

0.0
0.125
1.0
1.125
1.125
2.0
2.0
2.0
2.0
2.125
2.125
2.125
3.0
3.0
3.0
3.0
3.0
3.125
3.125
3.125
3.125
3.125
3.125
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.125
4.125
4.125
4.125
4.125
4.125
4.125
4.125
4.125
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.125
5.125
5.125
5.125
5.125
5.125
5.125
5.125
5.125
5.125
5.125
5.125
5.125
5.125
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.125
6.125
6.125
6.125
6.125
6.125
6.125
6.125
6.125
6.125
6.125
6.125
6.125
6.125
6.125
6.125
6.125
6.125
6.125
6.125
6.125
6.125
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
7.125
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0


In [6]:
using TensorOperations
Z=diagm(vcat(ones(20),-ones(20)))
function parity(tens::Array{T, 4}, Z) where {T}
	@tensor tens_flipped[-1, -2, -3, -4] := tens[1, 2, 3, 4] * Z[1, -1] * Z[2, -2] * Z[3, -3] * Z[4, -4]
	return dot(tens, tens_flipped) / (norm(tens))^2
end


parity (generic function with 4 methods)

In [7]:
real.(parity.(vecs[2:end],[Z,])).|>println;

-0.9999999999999996
1.0000000000000004
-0.9999999999999996
-0.9999999999999998
1.0000000000000004
0.9999999999999998
0.9999999999999998
1.0
-1.0
-0.9999999999999998
-1.0
0.9999999999999996
0.9999999999999998
0.9999999999999998
0.9999999999999998
0.9999999999999994
-1.0000000000000004
-0.9999999999999996
-1.0000000000000004
-0.9999999999999996
-1.0
-0.9999999999999996
0.9999999999999998
1.0000000000000002
1.0
0.9999999999999998
0.9999999999999994
1.0000000000000002
0.9999999999999998
1.0000000000000004
1.0000000000000004
-1.0
-1.0
-1.0
-0.9999999999999996
-0.9999999999999998
-0.9999999999999998
-0.9999999999999998
-0.9999999999999996
-1.0000000000000002
0.9999999999999998
1.0000000000000007
1.0
0.9999999999999998
0.9999999999999998
1.0000000000000002
1.0
0.9999999999999997
1.0000000000000004
1.0000000000000004
1.0
1.0000000000000004
1.0000000000000007
-0.9999999999999999
-0.9999999999999999
-0.9999999999999998
-1.0000000000000004
-0.9999999999999996
-1.0000000000000002
-1.0
-1.0
-0.9999

In [14]:
using Serialization
LDO_data=deserialize("DSO/rotate=true_30_6.0e-6_1.0e-10_rotate=(true)__path=chi=30_A_numerical.data");
sys=LDO_data["eigensystem"];
vals,vecs=sys[1],sys[2];
-log.(2,abs.(vals))[2:end] .|> println;

0.12485147077461711
0.9995286115549408
1.1240304376503514
1.1240304376503514
1.9969663740627837
1.9977501224984693
1.9981481904225595
1.9981481904225595
2.1188801644578166
2.121455952000827
2.124062036293034
2.9921064450459727
2.9921064450459727
2.9926744644928123
2.9964346846222822
2.998796363493067
3.1136514241571236
3.1136514241571236
3.120110996286751
3.120110996286751
3.1233994524890116
3.1233994524890116
3.988182153507063
3.988182153507063
3.9895829905994353
3.991236786760224
3.9934192250255705
3.9934192250255705
3.9998530335305964
3.9998530335305964
4.00019061231619
4.093031257180962
4.11480532286273
4.1164770977669765
4.119926679651086
4.123867590234653
4.124201973375091
4.124201973375091
4.126833952553666
4.142050658605244


In [15]:
parities=real.(parity.(vecs[2:end],[Z,]));

In [16]:
parities .|> println;

-1.0000000000000002
1.0
-1.0
-1.0
0.9999999999999999
1.0
1.0000000000000009
1.0000000000000009
-1.0000000000000007
-0.9999999999999991
-1.0000000000000002
1.0000000000000004
1.0000000000000004
1.0
1.0000000000000007
1.0
-1.0000000000000002
-1.0000000000000002
-1.0
-1.0
-0.9999999999999998
-0.9999999999999998
0.9999999999999999
0.9999999999999999
1.0000000000000009
1.0
0.9999999999999998
0.9999999999999998
1.0000000000000002
1.0000000000000002
1.0000000000000002
-1.0000000000000002
-0.9999999999999998
-1.0
-1.0000000000000004
-0.9999999999999996
-1.0000000000000002
-1.0000000000000002
-1.0000000000000002
-1.0000000000000004


In [17]:
signs=sign.(vals)[2:end]
println(signs)
for i=eachindex(parities)
    if parities[i]<0
        signs[i]*=-1
    end
end;
println(signs)

ComplexF64[-1.0 + 0.0im, 1.0 + 0.0im, 3.7712699761223354e-6 + 0.9999999999928887im, 3.7712699761223354e-6 - 0.9999999999928887im, -1.0 + 0.0im, -1.0 + 0.0im, -4.902391378014862e-5 + 0.9999999987983279im, -4.902391378014862e-5 - 0.9999999987983279im, 1.0 + 0.0im, 1.0 + 0.0im, -1.0 + 0.0im, 0.0001598226841101622 + 0.9999999872283548im, 0.0001598226841101622 - 0.9999999872283548im, -1.0 + 0.0im, -1.0 + 0.0im, 1.0 + 0.0im, -0.00025711603660101535 + 0.9999999669456713im, -0.00025711603660101535 - 0.9999999669456713im, -0.00011166584958351303 + 0.999999993765369im, -0.00011166584958351303 - 0.999999993765369im, 0.00011745555364439527 + 0.9999999931020965im, 0.00011745555364439527 - 0.9999999931020965im, 0.99999999097143 + 0.00013437685824214124im, 0.99999999097143 - 0.00013437685824214124im, 1.0 + 0.0im, 1.0 + 0.0im, 0.0002849197578219135 + 0.9999999594103649im, 0.0002849197578219135 - 0.9999999594103649im, -0.00014395004653651518 + 0.9999999896391919im, -0.00014395004653651518 - 0.999999989

In [18]:
imag.(log.(signs))./(pi/2) .|>println

-0.0
0.0
-1.000002400865034
1.000002400865034
2.0
2.0
1.0000312095928439
-1.0000312095928439
-2.0
-2.0
-0.0
0.9998982537187895
-0.9998982537187895
2.0
2.0
0.0
-0.9998363148455036
0.9998363148455036
-0.9999289113121093
0.9999289113121093
-1.0000747745279963
1.0000747745279963
8.554696516303829e-5
-8.554696516303829e-5
0.0
0.0
0.9998186144461783
-0.9998186144461783
1.000091641446175
-1.000091641446175
0.0
-0.0
-0.0
-0.0
-2.0
-2.0
-1.9999234568566855
1.9999234568566855
-0.0
-0.0


40-element Vector{Nothing}:
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 ⋮
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing

In [14]:
res=py"get_initial_tensor_aniso"(Dict("Jratio"=>0.5681,"relT"=>1.1923,"symmetry_tensors"=>true));
res.to_ndarray()

/home/ebel/Codes/GILT_TNR_R/GiltTNR/GiltTNR2D_Ising_benchmarks.py:168: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return sinh(2*x*Jv)*sinh(2*x*Jh) - 1
/home/ebel/Codes/GILT_TNR_R/GiltTNR/GiltTNR2D_Ising_benchmarks.py:168: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return sinh(2*x*Jv)*sinh(2*x*Jh) - 1
/home/ebel/Codes/GILT_TNR_R/GiltTNR/GiltTNR2D_Ising_benchmarks.py:168: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return sinh(2*x*Jv)*sinh(2*x*Jh) - 1
/home/e

2×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 5.47305  0.0
 0.0      1.73542

[:, :, 2, 1] =
 0.0      2.59443
 1.29638  0.0

[:, :, 1, 2] =
 0.0      1.29638
 2.59443  0.0

[:, :, 2, 2] =
 1.73542  0.0
 0.0      1.47305